# Synaptome Simulation Example with OBI-One Form Logic

This notebook demonstrates how to run a synaptome simulation using the OBI-One form-based workflow, similar to the circuit simulation example.

In [1]:
from entitysdk import Client
from obi_auth import get_token
from pathlib import Path

from obi_notebook import get_projects
from obi_notebook import get_entities
import obi_one as obi
import os
import pandas as pd

token = get_token(environment="production", auth_mode="daf")
project_context = get_projects.get_projects(token)
db_client = Client(environment="production", project_context=project_context, token_manager=token)

Dropdown(description='Select:', options=(('Test', {'id': 'af313912-d277-4021-a612-7b3b9dfe7fef', 'name': 'Test…

In [2]:
input_root = "/Users/ilkankilic/Workspace/obi-one/examples/H_synaptome_simulations/outputs/synaptomes_sscx_decontainerized/initialize.circuit=cell_003162__cADpyr_3"
output_root = "./obi-output/extracted_synaptome_circuits"

In [3]:
circ_path = f"{input_root}/circuit_config.json"
synaptome = obi.Circuit(name="cell_003162__cADpyr_3", path=circ_path)
print(f"Synaptome with {synaptome.sonata_circuit.edges.size} synapses")
print(f"Default node population: '{synaptome.default_population_name}'")

Synaptome with 29 synapses
Default node population: 'S1nonbarrel_neurons'


In [5]:
from pathlib import Path
import obi_one as obi

# === Parameters ===
sim_duration = 3000.0

sim_form = obi.SynaptomeSimulationScanConfig.empty_config()

# Info
info = obi.Info(
    campaign_name="Synaptome Simulation",
    campaign_description="Simulation of example synaptome"
)
sim_form.set(info, name="info")


# Timestamps
timestamps = obi.RegularTimestamps(start_time=0.0, number_of_repetitions=1, interval=100)
sim_form.add(timestamps, name="Timestamps")

all_neurons = obi.AllNeurons()
sim_form.add(all_neurons, name="AllNeurons")

# Stimulus
stimulus = obi.PoissonSpikeStimulus(
    duration=800.0,
    timestamps=timestamps.ref,
    frequency=20,
)
sim_form.add(stimulus, name="PoissonInput")

stimulus = obi.ConstantCurrentClampSomaticStimulus(
    timestamps=timestamps.ref, duration=2000.0, amplitude=0.5
)
sim_form.add(stimulus, name="CurrentClampInput")

# Recording
recording = obi.SomaVoltageRecording()
sim_form.add(recording, name="SomaVoltage")

# Initialization block
init = obi.SynaptomeSimulationScanConfig.Initialize(
    circuit=synaptome,
    simulation_length=sim_duration,
)
sim_form.set(init, name="initialize")
# Validated Config
validated_sim_conf = sim_form.validated_config()

print(validated_sim_conf)

# === 2. Wrap into a Simulation ===
grid_scan = obi.GridScanGenerationTask(form=validated_sim_conf, coordinate_directory_option="ZERO_INDEX", output_root='./obi-output/run_circuit_simulations/grid_scan')
grid_scan.execute()
obi.run_tasks_for_generated_scan(grid_scan)

SynaptomeSimulationScanConfig(type='SynaptomeSimulationScanConfig', timestamps={'Timestamps': RegularTimestamps(type='RegularTimestamps', start_time=0.0, interval=100.0, number_of_repetitions=1)}, recordings={'SomaVoltage': SomaVoltageRecording(type='SomaVoltageRecording', neuron_set=None, dt=0.1)}, info=Info(type='Info', campaign_name='Synaptome Simulation', campaign_description='Simulation of example synaptome'), neuron_sets={'SynaptomeCell': AllNeurons(type='AllNeurons', sample_percentage=100.0, sample_seed=1), 'AllNeurons': AllNeurons(type='AllNeurons', sample_percentage=100.0, sample_seed=1)}, initialize=Initialize(type='SynaptomeSimulationScanConfig.Initialize', circuit=Circuit(name='cell_003162__cADpyr_3', path='/Users/ilkankilic/Workspace/obi-one/examples/H_synaptome_simulations/outputs/synaptomes_sscx_decontainerized/initialize.circuit=cell_003162__cADpyr_3/circuit_config.json', matrix_path=None, type='Circuit'), simulation_length=3000.0, extracellular_calcium_concentration=1.

In [6]:
simulation_config_path = grid_scan.single_configs[0].coordinate_output_root / "simulation_config.json"
print(simulation_config_path)

circuit_folder = grid_scan.single_configs[0].coordinate_output_root / "sonata_circuit"

obi-output/run_circuit_simulations/grid_scan/0/simulation_config.json


In [ ]:
# Remove the old compiled mod files folder
! rm -r arm64/
# flag DISABLE_REPORTINGLIB to skip SonataReportHelper.mod and SonataReport.mod from compilation.
!../../.venv/bin/nrnivmodl -incflags "-DDISABLE_REPORTINGLIB" {circuit_folder}/mod

In [ ]:
# === 4. Run the simulation (BlueCelluLab backend) ===
from obi_one.scientific.library.simulation_execution import run

run(
    simulation_config=simulation_config_path,
    simulator='bluecellulab',
    save_nwb=False
)

## Results
The results are stored in the `output` directory. You can analyze the voltage traces and other outputs as needed.

In [9]:
import bluepysnap
snap_simulation = bluepysnap.Simulation(simulation_config_path)
spikes = snap_simulation.spikes
print(
    spikes.time_start,
    spikes.time_stop,
    spikes.dt
)
print(spikes.population_names)

0 3000.0 0.025
['S1nonbarrel_neurons']


In [10]:
population_name = synaptome.default_population_name

spike_pop = spikes[population_name]
node_population = spike_pop.nodes
filtered = spikes.filter( t_start=spikes.time_start, t_stop=spikes.time_stop)
filtered.report.head()

,ids,population
times,,
1.000000e-10,0,S1nonbarrel_neurons
1.227500e+01,0,S1nonbarrel_neurons
2.275000e+01,0,S1nonbarrel_neurons
3.327500e+01,0,S1nonbarrel_neurons
4.407500e+01,0,S1nonbarrel_neurons


In [11]:
snap_simulation.reports

{'SomaVoltage': <bluepysnap.frame_report.SomaReport at 0x33fb6e8a0>}

In [12]:
soma_report = snap_simulation.reports['SomaVoltage']
print(
    soma_report.time_start,
    soma_report.time_stop,
    soma_report.dt
)  # Gives a warning in case the dt differs from simulation.dt

[2025-10-17 16:44:11,439] WARNING: dt from the report differs from the global simulation dt.
0.0 3000.0 0.1
